# Setup

In [1]:
# Load packages 
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [4]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "project"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import col
sqlContext = SQLContext(sc)

# Read in data 

In [5]:
df = sqlContext.read.format("csv") \
               .option("inferSchema", "true") \
               .option("header", "false") \
               .option("delimiter", "\t")\
               .load("sample_training.txt") #Adjust path as needed

df = df.withColumnRenamed("_c0", "CTR") \
       .withColumnRenamed("_c1", "NumVar1") \
       .withColumnRenamed("_c2", "NumVar2") \
       .withColumnRenamed("_c3", "NumVar3") \
       .withColumnRenamed("_c4", "NumVar4") \
       .withColumnRenamed("_c5", "NumVar5") \
       .withColumnRenamed("_c6", "NumVar6") \
       .withColumnRenamed("_c7", "NumVar7") \
       .withColumnRenamed("_c8", "NumVar8") \
       .withColumnRenamed("_c9", "NumVar9") \
       .withColumnRenamed("_c10", "NumVar10") \
       .withColumnRenamed("_c11", "NumVar11") \
       .withColumnRenamed("_c12", "NumVar12") \
       .withColumnRenamed("_c13", "NumVar13") \
        .withColumnRenamed("_c14", "NumVar14") \
        .withColumnRenamed("_c15", "NumVar15") \
        .withColumnRenamed("_c16", "NumVar16") \
        .withColumnRenamed("_c17", "NumVar17") \
        .withColumnRenamed("_c18", "NumVar18") \
        .withColumnRenamed("_c19", "NumVar19") \
        .withColumnRenamed("_c20", "NumVar20") \
        .withColumnRenamed("_c21", "NumVar21") \
        .withColumnRenamed("_c22", "NumVar22") \
        .withColumnRenamed("_c23", "NumVar23") \
        .withColumnRenamed("_c24", "NumVar24") \
        .withColumnRenamed("_c25", "NumVar25") \
        .withColumnRenamed("_c26", "NumVar26") \
        .withColumnRenamed("_c27", "NumVar27") \
        .withColumnRenamed("_c28", "NumVar28") \
        .withColumnRenamed("_c29", "NumVar29") \
        .withColumnRenamed("_c30", "NumVar30") \
        .withColumnRenamed("_c31", "NumVar31") \
        .withColumnRenamed("_c32", "NumVar32") \
        .withColumnRenamed("_c33", "NumVar33") \
        .withColumnRenamed("_c34", "NumVar34") \
        .withColumnRenamed("_c35", "NumVar35") \
        .withColumnRenamed("_c36", "NumVar36") \
        .withColumnRenamed("_c37", "NumVar37") \
        .withColumnRenamed("_c38", "NumVar38") \
        .withColumnRenamed("_c39", "NumVar39")


#df.select("*").show(5)
#df.select("CTR", "NumVar1", "NumVar2", "NumVar3", "NumVar4", "NumVar5", "NumVar6", "NumVar7", "NumVar8", "NumVar9", "NumVar10", "NumVar11", "NumVar12", "NumVar13").show()




In [8]:
subset_ctr1 = df.filter(col("CTR") == 1) #get all of the CTR == 1 

In [9]:
# convert to RDD 
subset_ctr1_rdd = subset_ctr1.rdd

In [29]:
# full rdd 
full_rdd = df.rdd

In [39]:
# Helper function 
def newvalues(x, broadcast_variable): 
    if x in broadcast_variable: #if the value is within our 'most frequent' list, keep the value
        yield(x) 
    else: #else, input None
        yield(None) #Note to self: Perhaps 'None' is not ideal if we want to input a null value for all other non-frequent values? 

In [45]:
#####
# Make the most common values for each column that we care about. 
# These 'most common' values cover 75% of our Y = 1 
# 75% is arbitrary 
######


count_1 = subset_ctr1_rdd.count() #total count of Y=1 in sample 
columns = ['NumVar20','NumVar28'] #testing with a small subset of columns 
holder = []
for c in columns: #There has to be a better way of applying functions to columns...but not an RDD? 
    print(c)
    # Count up the frequencies of each key and sum, dividing by total number of Y=1 
    cumulsum_df = pd.DataFrame(subset_ctr1_rdd.map(lambda x: (x[c],1)).reduceByKey(lambda x,y: (x+y)/count_1).collect())
    cumulsum_df.columns = ['value','pct']
    cumulsum_df['cumsum']=cumulsum_df['pct'].cumsum()
    # specify some majority threshold (in this case, 0.75)
    cutoff_index = max(cumulsum_df[cumulsum_df['cumsum']>=0.75].index)
    mostfreq = list(cumulsum_df.loc[0:cutoff_index]['value'].values) #these are the most frequent values in the column 
    mostfreq_bc = sc.broadcast(mostfreq) #broadcast most frequent values. 
    modified_column = full_rdd.flatMap(lambda x: newvalues(x[c], mostfreq_bc.value)).collect()
    holder.append(modified_column)

NumVar20
NumVar28


KeyboardInterrupt: 

In [51]:
pd.DataFrame(holder) #this takes forever, and I don't know why??? 

['8fb24933',
 '3f4ec687',
 '65c53f25',
 'e64ca89e',
 '675e81f6',
 '875679ca',
 '970f01b2',
 '7956c2ff',
 'def4a4d4',
 '38eb9cf4',
 '869243b9',
 '4c5c7066',
 '49042125',
 '49b74ebc',
 '01a0f3f6',
 '5c116cc3',
 '24db17b4',
 '9e5b069d',
 '2b322b66',
 '7fafef37',
 'ea98b75d',
 'f00bddf8',
 'f93c8e47',
 'd2dbdfe6',
 'f1ff45d6',
 '78f06ef9',
 'ce8217f8',
 '468a0854',
 '928efe34',
 '24349845',
 'b3b280b5',
 '2ba364b2',
 '6284da2d',
 '48db53e5',
 'd385ea68',
 'cdd4e327',
 '3598a741',
 'e7698644',
 'daf51ef3',
 'e2ec9176',
 'ce80bb26',
 '5ce839a9',
 '0f59d328',
 '3cf1ccde',
 '40635b15',
 '6de90931',
 '0492c809',
 '50b436c9',
 'ef1eacfd',
 '5f29da0e',
 'b920f7a4',
 'f8008800',
 'b13433e4',
 '31be2dc6',
 '03db5ecd',
 '57b4bd89',
 'd5b22f0b',
 '6746bfa7',
 '26a81064',
 '56fb669f',
 'bc324536',
 '728ec517',
 '11754474',
 '4bca059f',
 'f1ff45d6',
 'd20b4953',
 'd385ea68',
 '09636619',
 'ad3508b1',
 'b3e8889a',
 'f14f1abf',
 '50db917d',
 '6772d022',
 'ec258437',
 'e14874c9',
 '6284da2d',
 '95402f9a',